In [114]:
import numpy as np
import cv2
def sort_contours(cnts, method="left-to-right"):
    # initialize the reverse flag and sort index
    reverse = False
    i = 0
  
    # handle if we need to sort in reverse
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True
  
    # handle if we are sorting against the y-coordinate rather than
    # the x-coordinate of the bounding box
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1
  
    # construct the list of bounding boxes and sort them from top to
    # bottom
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
        key=lambda b:b[1][i], reverse=reverse))
  
    # return the list of sorted contours and bounding boxes
    return (cnts)
def find_words(img):
    #make a list of cropped words
    words=[]
    # Convert the image to gray scale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Performing OTSU threshold
    ret, thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
    # Specify structure shape and kernel size.
    # Kernel size increases or decreases the area
    # of the rectangle to be detected.
    # A smaller value like (10, 10) will detect
    # each word instead of a sentence.
    a=img.shape[1]*0.025
    b=img.shape[0]*0.025
    x=(a+b)/2
    x=int(x)
    rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (x, x))
    # Applying dilation on the threshold image
    dilation = cv2.dilate(thresh1, rect_kernel, iterations = 1)
    # Finding contours
    contours, hierarchy = cv2.findContours(dilation, cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    #sorting contours in x-axis
    # Creating a copy of image
    im2 = img.copy()
    # Looping through the identified contours
    # Then rectangular part is cropped and passed on
    # to pytesseract for extracting text from it
    # Extracted text is then written into the text file
    area=img.shape[0]*img.shape[1]
    contours=sort_contours(contours,method="right-to-left")
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        # Drawing a rectangle on copied image
        c_area=w*h
        if c_area > (area*0.01) and (area*0.2) > c_area:
            rect = cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2)
            # Cropping the text block for giving input to the model
            cropped = im2[y:y + h, x:x + w]
            words.append(cropped)
        #showing the detetected image
    cv2.imshow('a',rect)
    cv2.waitKey(0)
    return(words)
def find(path):
    #make a list of cropped words
    lines=[]
    img = cv2.imread(path)
    # Convert the image to gray scale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Performing OTSU threshold
    ret, thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
    
    # Specify structure shape and kernel size.
    # Kernel size increases or decreases the area
    # of the rectangle to be detected.
    # A smaller value like (10, 10) will detect
    # each word instead of a sentence.
    a=img.shape[1]*0.0663
    b=img.shape[0]*0.0663
    x=(a+b)/2
    x=int(x)
    print(x)
    rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (x, x))
    # Applying dilation on the threshold image
    dilation = cv2.dilate(thresh1, rect_kernel, iterations = 1)
    # Finding contours
    contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    #sorting contours in y-axis
    #contours.sort(key=lambda x:get_precedence(y, img.shape[0]))
    # Creating a copy of image
    im2 = img.copy()

    # Looping through the identified contours
    # Then rectangular part is cropped and passed on
    # to pytesseract for extracting text from it
    # Extracted text is then written into the text file
    a=0
    contours=sort_contours(contours,method="up-to-bottom")
    for i in range(len(contours)):
        x, y, w, h = cv2.boundingRect(contours[-(i+1)])
        # Drawing a rectangle on copied image
        rect = cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2)
        # Cropping the text block for giving input to the model
        cropped = im2[y:y + h, x:x + w]
        lines.append(cropped)
        #showing the detetected image
        #cv2.imwrite((str(a)+'.jpeg'),cropped)
        a=a+1
    all_photo=[]
    for i in range(len(contours)):
        f=find_words(lines[i])
        all_photo.append(f)
        print(i)
    cv2.imshow('a',rect)    
    cv2.waitKey(0)
    return(all_photo)